## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import google_api_key

# Configure gmaps API key
gmaps.configure(api_key=google_api_key)


In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nanortalik,GL,60.1432,-45.2371,30.60,87,90,16.64,overcast clouds
1,1,Tasiilaq,GL,65.6145,-37.6368,26.60,46,0,9.22,clear sky
2,2,Suzaka,JP,36.6500,138.3167,30.99,71,94,7.00,light snow
3,3,Lasa,IT,46.6166,10.7002,39.00,93,90,6.91,light snow
4,4,Cape Town,ZA,-33.9258,18.4232,66.20,68,0,14.97,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Hilo,US,19.7297,-155.0900,82.40,47,20,14.97,few clouds
10,10,Cockburn Town,TC,21.4612,-71.1419,77.00,83,20,6.91,few clouds
15,15,Maxixe,MZ,-23.8597,35.3472,77.88,87,99,6.13,overcast clouds
17,17,Georgetown,MY,5.4112,100.3354,80.01,83,20,2.30,few clouds
18,18,San Policarpo,PH,12.1791,125.5072,79.72,86,100,20.02,overcast clouds
20,20,Mahajanga,MG,-15.7167,46.3167,77.00,94,40,4.61,scattered clouds
21,21,Rikitea,PF,-23.1203,-134.9692,77.61,71,81,12.53,broken clouds
24,24,Puerto Ayora,EC,-0.7393,-90.3518,77.00,96,93,4.00,overcast clouds
33,33,Carnarvon,AU,-24.8667,113.6333,77.00,100,7,8.05,clear sky
48,48,Hithadhoo,MV,-0.6000,73.0833,81.88,69,34,8.34,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                187
City                   187
Country                186
Lat                    187
Lng                    187
Max Temp               187
Humidity               187
Cloudiness             187
Wind Speed             187
Current Description    187
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                186
City                   186
Country                186
Lat                    186
Lng                    186
Max Temp               186
Humidity               186
Cloudiness             186
Wind Speed             186
Current Description    186
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Hilo,US,82.40,few clouds,19.7297,-155.0900,
10,Cockburn Town,TC,77.00,few clouds,21.4612,-71.1419,
15,Maxixe,MZ,77.88,overcast clouds,-23.8597,35.3472,
17,Georgetown,MY,80.01,few clouds,5.4112,100.3354,
18,San Policarpo,PH,79.72,overcast clouds,12.1791,125.5072,
20,Mahajanga,MG,77.00,scattered clouds,-15.7167,46.3167,
21,Rikitea,PF,77.61,broken clouds,-23.1203,-134.9692,
24,Puerto Ayora,EC,77.00,overcast clouds,-0.7393,-90.3518,
33,Carnarvon,AU,77.00,clear sky,-24.8667,113.6333,
48,Hithadhoo,MV,81.88,scattered clouds,-0.6000,73.0833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_api_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
    print(index)
        

6
10
15
17
18
20
21
24
33
48
51
54
57
59
69
71
74
79
81
85
100
104
109
110
112
113
115
121
128
130
132
133
140
142
147
149
159
168
175
176
177
183
187
188
192
195
196
197
198
203
205
207
210
213
216
217
219
225
227
234
236
243
244
258
261
267
Hotel not found... skipping.
273
277
280
291
292
294
297
298
Hotel not found... skipping.
316
320
321
324
325
331
337
342
343
344
347
352
361
365
369
370
376
378
380
383
390
391
397
398
406
414
420
421
423
426
427
430
431
436
438
439
440
447
448
452
454
455
463
476
482
484
485
489
490
493
494
499
501
505
511
517
522
524
529
539
541
546
552
553
555
556
560
565
566
573
574
580
581
584
585
588
589
593
595
598
599
605
608
613
617
631
640
642
644
645
649
657
659
661
664
Hotel not found... skipping.
671
673
675
Hotel not found... skipping.
677
678
680
683
684
685
686
698
Hotel not found... skipping.
699
705
710
711
713
716


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Hilo,US,82.40,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
10,Cockburn Town,TC,77.00,few clouds,21.4612,-71.1419,Osprey Beach Hotel
15,Maxixe,MZ,77.88,overcast clouds,-23.8597,35.3472,Oceano
17,Georgetown,MY,80.01,few clouds,5.4112,100.3354,Cititel Penang
18,San Policarpo,PH,79.72,overcast clouds,12.1791,125.5072,JM Lodge
20,Mahajanga,MG,77.00,scattered clouds,-15.7167,46.3167,Restaurant Greedy Coco Lodge Majunga
21,Rikitea,PF,77.61,broken clouds,-23.1203,-134.9692,Pension Maro'i
24,Puerto Ayora,EC,77.00,overcast clouds,-0.7393,-90.3518,Castillo Galapagos
33,Carnarvon,AU,77.00,clear sky,-24.8667,113.6333,Hospitality Carnarvon
48,Hithadhoo,MV,81.88,scattered clouds,-0.6000,73.0833,Scoop Guest House


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))